## Project Goal

The goal of this project is to develop a **Product Recommendation System** that suggests products (e.g., wine, fruits, meat) based on past purchase behavior. By leveraging collaborative filtering and matrix factorization techniques, the project aims to:

1. Recommend personalized products to customers to enhance their shopping experience.
2. Identify patterns in purchase behavior to inform cross-selling strategies.
3. Evaluate the quality of recommendations using precision, recall, and F1-score metrics.

## Libraries

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
import scipy.sparse as sp
from sklearn.decomposition import TruncatedSVD

In [2]:
#Mount Google Drive
drive.mount('/content/drive')

#Define the file path
file_path = '/content/drive/My Drive/marketing_campaign.csv'

#Load the dataset
df = pd.read_csv(file_path, delimiter='\t')
print(df.head())

Mounted at /content/drive
     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...                  7             0   
1  08-03-2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   
3  10-02-2014       26        11  ...                  6             0   
4  19-01-2014       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  C

## Recommendation System

In [3]:
# Preprocessing: Select relevant purchase data
purchase_features = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
user_item_matrix = df[purchase_features]
user_item_matrix.index = df['ID']  # Assuming 'ID' is a unique customer identifier

# Normalize the data (optional: to account for different scales)
user_item_matrix = user_item_matrix.div(user_item_matrix.sum(axis=1), axis=0).fillna(0)

# Convert to a sparse matrix for efficient computation
sparse_matrix = sp.csr_matrix(user_item_matrix.values)

# Apply Matrix Factorization using Truncated SVD
n_components = 5  # Number of latent factors
svd = TruncatedSVD(n_components=n_components, random_state=42)
user_factors = svd.fit_transform(sparse_matrix)
item_factors = svd.components_.T

# Compute similarity scores (item-based)
item_similarity = cosine_similarity(item_factors)

# Recommendation function
def recommend_products(customer_id, top_n=3):
    user_index = user_item_matrix.index.get_loc(customer_id)
    user_purchases = sparse_matrix[user_index, :].toarray().flatten()
    scores = user_purchases @ item_similarity  # Weighted sum of similarities
    scores[user_purchases > 0] = 0  # Exclude already purchased products
    recommended_items = np.argsort(scores)[::-1][:top_n]
    return user_item_matrix.columns[recommended_items]

# Example: Recommend top 3 products for a specific customer
customer_id = user_item_matrix.index[0]  # Replace with a valid customer ID
recommendations = recommend_products(customer_id, top_n=3)
print(f"Recommended products for customer {customer_id}: {recommendations.tolist()}")

# Evaluate the recommendation system
# Create a simple train-test split (e.g., 80-20 split)
def evaluate_recommendations():
    train_matrix = sparse_matrix[:int(0.8 * sparse_matrix.shape[0]), :]
    test_matrix = sparse_matrix[int(0.8 * sparse_matrix.shape[0]):, :]

    # Predict for all users in the test set
    precision_list, recall_list, f1_list = [], [], []
    for user_index in range(test_matrix.shape[0]):
        user_purchases = test_matrix[user_index, :].toarray().flatten()
        scores = user_purchases @ item_similarity
        scores[user_purchases > 0] = 0
        recommended_items = np.argsort(scores)[::-1][:5]  # Top 5 recommendations

        true_positives = np.isin(recommended_items, np.where(user_purchases > 0)[0]).sum()
        precision = true_positives / len(recommended_items)
        recall = true_positives / (user_purchases > 0).sum()

        precision_list.append(precision)
        recall_list.append(recall)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_list.append(f1)

    print(f"Precision: {np.mean(precision_list):.2f}")
    print(f"Recall: {np.mean(recall_list):.2f}")
    print(f"F1-Score: {np.mean(f1_list):.2f}")

# Evaluate the system
evaluate_recommendations()

Recommended products for customer 5524: ['MntGoldProds', 'MntSweetProducts', 'MntFishProducts']
Precision: 0.92
Recall: 0.85
F1-Score: 0.87


### Explanation:
1. **Collaborative Filtering**:
   - Built an item-based collaborative filtering model using cosine similarity on product latent factors.

2. **Matrix Factorization**:
   - Applied Truncated SVD to decompose the user-item matrix into latent factors for users and items.

3. **Evaluation**:
   - Used precision, recall, and F1-score to evaluate the quality of recommendations.

4. **Recommendations**:
   - The `recommend_products` function returns the top `n` product recommendations for a given customer.

---

### Results Interpretation
#### Example Recommendations
- **Customer ID: 5524**
  - Recommended Products: `['MntGoldProds', 'MntSweetProducts', 'MntFishProducts']`
  - Interpretation: The system successfully recommends products that the customer has not purchased but are likely to align with their purchase patterns based on similarity.

#### Evaluation Metrics
- **Precision: 0.92**
  - High precision indicates that 92% of the recommended products were relevant (true positives).

- **Recall: 0.85**
  - The model captured 85% of all relevant products for the users in the test set.

- **F1-Score: 0.87**
  - The harmonic mean of precision and recall suggests a strong balance between accuracy and coverage of recommendations.

### Deliverables
1. A functional recommendation system that provides personalized product suggestions.
2. Insights from purchase patterns to inform cross-selling strategies.